In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from matplotlib.backends.backend_pdf import PdfPages
import scipy.cluster.hierarchy as spc
from scipy.stats import pearsonr

def getDataFrame():
    df = pd.read_csv("Stats_task2.csv")
    return df


class Data:
    def __init__(self, mz, rt, intensity):
        self.mz = mz
        self.rt = rt
        self.i = intensity
    
    def calculateMeanIntensity(self):
        self.meanIntensity = sum(self.i)/len(self.i)

    def calculateMedIntensity(self):
        self.medIntensity = np.median([x for x in self.i])
        
    def Annotate(self, string):
        self.anno = string


def getData():
    df = getDataFrame()
    dfValues = df.values.tolist()
    data = list()
    for i in range(len(dfValues)):
        data.append(Data(dfValues[i][1], dfValues[i][2], [j for j in dfValues[i][6:16]]))
    return data

def create_df(i,j):
    temp = list()
    col = list()
    for x in range(i,j):
        temp.append(data[x].i)
        col.append(x)
    df = pd.DataFrame(temp)
    df.set_axis(col, axis=0, inplace=True)
    return df.T



if __name__ == "__main__":
    data = getData()
    for i in range(len(data)):
        data[i].calculateMeanIntensity()
#     print(data[1].meanIntensity)
#     print(data[2].i)
    
    
    # Counting number of zero intensity 
    # and removing feature with greater than 5 zeroes
    i=0
    while(i<len(data)):
        cnt=0
        for j in range(0,10):
            if data[i].i[j]==0:
                cnt = cnt+1
        if cnt>3:
            del data[i]
        else:
            i = i+1
            
    
    #replace missing value with median intensity
    # and taking log transformation
    for i in range(len(data)):
        data[i].calculateMedIntensity()
        
    
    for i in range(len(data)):
        for j in range(0,10):
            if data[i].i[j] == 0:
                data[i].i[j] = data[i].medIntensity
                data[i].i[j] = np.log(data[i].i[j])
            else:
                data[i].i[j] = np.log(data[i].i[j])
                
    
#     #Isotope Detection            
#     def isotope(*ind):
#         rt_tol = 0.05
#         i = ind[0][0]
#         j = ind[-1][-1]
#         for x in range(0,j-i-1): 
#             for y in range(x+1,j-i):
#                 d1 = data[x].i
#                 d2 = data[y].i
                
#                 correlation = np.corrcoef(d1, d2)[0][1]

#                 if (1.003-0.01<= abs(data[x].mz-data[y].mz) <=1.003+0.01) and (abs(data[x].rt-data[x].rt)<rt_tol):
#                     print("Isotopes are m/z %d and %d and corr = %f" %(data[x].mz,data[y].mz, correlation)) 
                    
    #Calculating correalion matrix and forming clusters                
    def corr(*ind):
        i = ind[0][0]
        j = ind[-1][-1]
        df_small = create_df(i,j+1)
#         print(df_small)
        correlation_mat = df_small.corr()
#         sns.heatmap(correlation_mat, annot = True)
#         plt.show()
        corelat = correlation_mat .values
        pdist = spc.distance.pdist(corelat)
        linkage = spc.linkage(pdist, method='average')
        idx = spc.fcluster(linkage, 0.5 * pdist.max(), 'distance')
        print("Cluster - ",idx)
        print("Number of cluster = %d" %idx.max())
        
        col = df_small.columns
        for i in range(1,idx.max()+1):
            clust = []
            cnt = 0
            for j in range(0,len(idx)):
                if idx[j] == i:
                    clust.append(col[j])
                    cnt = cnt + 1;
            print("cnt = %d"%cnt)
            print(clust)
            
            if(cnt>1):
                maxi = 0
                for x in range(0,len(clust)):
                    if(data[clust[x]].medIntensity>maxi):
                        maxi = data[clust[x]].medIntensity
                        maxind = clust[x]
                data[maxind].Annotate(['M-H'])
                print(data[maxind].anno)

                neutral_mass = data[maxind].mz+1.007

                #Test for dimer present
                dimer_mass = 2*neutral_mass-1.007
                for x in range(0,len(clust)):
                    if(data[clust[x]].mz==dimer_mass):
                        data[clust[x]].Annotate(['2M-H'])
                        print(data[clust[x]].anno)
                        
                # Test for annotation file
                for i in range(len(Ano_mass)):
                    if Ano_charge[i]==0:
                        Ano_charge[i] = 1
                    cal_mz = (neutral_mass + Ano_mass[i])/(np.abs(Ano_charge[i]))
                    for x in range(0,len(clust)):
                        if(data[clust[x]].mz==cal_mz):
                            data[clust[x]].Annotate(Ano[i])
                            print(data[clust[x]].anno)        
                
    #Binning
    rt_tol = 0.03
    j = 1
    ind = []
    mass = []
    for i in range(len(data)-1000):
        if data[j].rt - data[i].rt >= rt_tol:
#             print("Bin - ",ind)
#             print("Binmass -",mass)
            ind.append(i)
            mass.append(data[i].mz)
            print("Bin - ",ind)
            print("Binmass -",mass)
            if len(ind)>1:
                corr(ind)
#                 isotope(ind)
            ind = []
            mass = []
        else:
            ind.append(i)
            mass.append(data[i].mz)
        j = j+1
        

Bin -  [0, 1, 2]
Binmass - [201.028, 301.029, 343.041]
Cluster -  [3 1 2]
Number of cluster = 3
cnt = 1
[1]
cnt = 1
[2]
cnt = 1
[0]
Bin -  [3, 4, 5, 6, 7]
Binmass - [283.031, 365.036, 447.04, 403.003, 693.053]
Cluster -  [1 1 1 2 1]
Number of cluster = 2
cnt = 4
[3, 4, 5, 7]
['M-H']
cnt = 1
[6]
Bin -  [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44]
Binmass - [529.043, 798.054, 141.019, 715.046, 469.033, 552.042, 387.029, 470.036, 486.012, 551.039, 649.016, 701.032, 797.051, 634.046, 813.026, 455.018, 633.043, 732.023, 305.027, 373.014, 388.033, 485.009, 537.023, 568.015, 650.019, 731.021, 828.998, 895.029, 567.012, 635.001, 746.995, 500.981, 664.992, 582.986, 762.97, 191.075, 418.978]
Cluster -  [2 3 3 3 2 3 2 3 1 2 2 5 3 3 3 4 2 1 2 3 3 3 4 3 3 2 3 3 2 3 2 3 3 3 3 3 3]
Number of cluster = 5
cnt = 2
[16, 25]
['M-H']
cnt = 10
[8, 12, 14, 17, 18, 24, 26, 33, 36, 38]
['M-H']
cnt = 22
[9, 10,

In [68]:
for i in range(len(data)):
    data[i].calculateMedIntensity()

In [39]:
data[1].mz

301.029

In [5]:
data[0].medIntensity

2761.9

In [8]:
f = open("anno.txt", "r")
print(f.read())

Annotation	Mass	Mode	Charge	Tier
H	1.007276	Negative	-1	1
2H	2.014552	Negative	-2	1
3H	3.021828	Negative	-3	1
NH4+	18.033823	Negative	-1	2
Na	22.989218	Negative	-1	1
H+Na	23.996494	Negative	-2	1
2H+Na	25.003770	Negative	-3	1
K	38.963158	Negative	-1	2
H+K	39.970434	Negative	-2	2
2H+K	40.977710	Negative	-3	2
2Na	45.978436	Negative	-2	1
H+2Na	46.985712	Negative	-3	1
Na+K	61.952376	Negative	-2	2
H+Na+K	62.959652	Negative	-3	2
3Na	68.967654	Negative	-3	1
2K	77.926316	Negative	-2	2
H+2K	78.933592	Negative	-3	2
2Na+K	84.941594	Negative	-3	2
Na+2K	100.915534	Negative	-3	2
3K	116.889474	Negative	-3	2
H	-1.007276	Negative	-1	1
Cl-	34.969402	Negative	-1	1
COOH-	44.998201	Negative	-1	1
H2O+CO	-46.005479	Both	0	1
NH3+H2O	-35.037114	Both	0	1
H2O	-18.010565	Both	0	1
NH3	-17.026549	Both	0	1
Na-H	21.981945	Both	0	1
MeOH	32.026215	Both	0	1
K-H	37.955882	Both	0	1
Acetonitrile	41.026549	Both	0	1
2Na-2H	43.963890	Both	0	1
HCOOH	46.005479	Both	0	1
NaCl	57.958622	Both	0	1
3Na-3H	65.945835	Both	0	1
NaCOOH	67.

In [31]:
with open(r"anno.txt", "r+") as f: 
    data = f.readlines() 
    Ano_mass=[]
    Ano_charge = []
    Ano = []
for x in data:
    Ano_mass.append(x.split('\t')[1])
    Ano_charge.append(x.split('\t')[3])
    Ano.append(x.split('\t')[0])
f.close()

In [32]:
Ano_mass.pop(0)
Ano_charge.pop(0)
Ano.pop(0)

'Annotation'

In [11]:
mass = np.array(mass)
m1 = mass[0:23]
charge = np.array(charge) 
c1 = charge[0:23]
anno = np.array(anno)
m1 = m1.astype('float')
c1 = c1.astype('float')
mass = mass.astype('float')

In [12]:
mz1 = np.divide(m1,c1)
len(mz1)

23

In [13]:
mz = np.append(mz1,mass[23:42])

In [40]:
Ano_mass = np.array(Ano_mass)
Ano_mass = Ano_mass.astype('float')
Ano_charge = np.array(Ano_charge) 
Ano_charge = Ano_charge.astype('float')